# ConvNeXt论文导读与模型精讲

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_38_ConvNeXt论文导读与模型精讲：

https://www.bilibili.com/video/BV1iP4y1M7nk/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

***论文***

A ConvNet for the 2020s：

https://arxiv.org/pdf/2201.03545.pdf

Deep Residual Learning for Image Recognition：

https://arxiv.org/pdf/1512.03385.pdf

## ConvNeXt论文

* 理解什么是ResNet-50
  * 由48层卷积+1层maxpool+1层avgpool构成，卷积每个block的配比为3：4：6：3
    
**ResNet50 Architecture**

![](./img/P38_1.png)

* ConvNeXt主要宗旨
  * 本文主要是希望基于ResNet-50结构，并参考Swin-T中的思想来升级改造ResNet，最终得到ResNeXt结构，并实现了新的准确率，并进一步探索了它的可扩展性。
* 优化器参数
  * AdamW，300epochs
  * 准确率直接从76.1%提升到了78.8%
  * 预训练学习率为4e-3，weight_decay=0.05，batchsize=4096
  * 微调学习率为5e-5，weight_decay=1e-8，batchsize=512，layer-wise lr decay

* 宏观设计
  * 将[3, 4, 6, 3]的区块比例改成了[3, 3, 9, 3]
  * 将底层的卷积替换成了4*4 stride=4的卷积，类似于patch
  * 引入depth-wise conv，并将channels从64提升到96 （群卷积：将输入通道划分为不同的group。 深度可分离卷积：有多少输入通道就划分多少个group）
  * 引入bottleneck结构 {channel 分别为 96 384 96}，并增大kernel size到7*7
  * 至此，ImageNet-1K的准确率从78.8%提升到80.6%
* 微观设计
  * 将RELU替换成GELU，将BN替换成LN
  * 引入更少激活函数和归一化层
  * 采用2 * 2，stride=2的卷积进行下采样，并在底层、下采样之前和最后的平均池化之后加入LN层，使得训练更加稳定
  * 至此，ImageNet-1K的准确率进一步提升到82.0%，击败Swin-T中的81.3%
  

* 可扩展性
  * ImageNet-1K训练
    * 随着参数数目和计算量的增大，准确率也在逐步提升至85.5%
  * 增加ImageNet-22k预训练，再迁移至ImageNet-1K微调
    * 伴随预训练，同样的模型，效果涨幅约为2%
    * 最终，ConvNeXt-XL效果达到87.8%